In [1]:
import httpx
import pandas as pd
from pydantic import BaseModel, model_validator
from tqdm.asyncio import tqdm_asyncio
from tqdm.auto import tqdm

In [2]:
df = pd.read_csv("../data/bare_records.csv")
df.shape

(814225, 3)

In [3]:
df = df.dropna()
df.shape

(814225, 3)

In [4]:
df.head()

,object_id,datestamp,set_spec
0,https://id.rijksmuseum.nl/2001,2025-01-14T19:57:56Z,['26051']
1,https://id.rijksmuseum.nl/20010,2025-01-14T19:34:32Z,['26051']
2,https://id.rijksmuseum.nl/20010000,2025-01-14T12:12:07Z,"['260239', '26051']"
3,https://id.rijksmuseum.nl/200100000,2025-01-16T21:45:21Z,"['260106', '260239']"
4,https://id.rijksmuseum.nl/200100001,2025-01-16T21:45:22Z,"['260106', '260239']"


In [10]:
with httpx.Client() as client:
    response = client.get(df.object_id.iloc[0])
    data = response.json()
    data

{'@context': 'https://linked.art/ns/v1/linked-art.json',
 'id': 'https://id.rijksmuseum.nl/2001',
 'type': 'HumanMadeObject',
 'produced_by': {'type': 'Production',
  'technique': [{'id': 'https://id.rijksmuseum.nl/220922',
    'type': 'Type',
    'identified_by': [{'type': 'Identifier',
      'assigned_by': [{'type': 'AttributeAssignment',
        'assigned': ['https://id.rijksmuseum.nl/2001'],
        'assigned_property': 'influenced_by'}],
      'content': '3',
      'classified_as': [{'id': 'http://vocab.getty.edu/aat/300456575',
        'type': 'Type'}]}]},
   {'id': 'https://id.rijksmuseum.nl/2202942',
    'type': 'Type',
    'identified_by': [{'type': 'Identifier',
      'assigned_by': [{'type': 'AttributeAssignment',
        'assigned': ['https://id.rijksmuseum.nl/2001'],
        'assigned_property': 'influenced_by'}],
      'content': '1',
      'classified_as': [{'id': 'http://vocab.getty.edu/aat/300456575',
        'type': 'Type'}]}]},
   {'id': 'https://id.rijksmuseum.nl/22

In [ ]:
data.keys()

dict_keys(['@context', 'id', 'type', 'produced_by', 'subject_of', 'assigned_by', 'identified_by', 'classified_as', 'dimension', 'made_of', 'shows', 'referred_to_by', 'equivalent', 'member_of'])

In [64]:
data['referred_to_by'][6]['classified_as'][0]['id']

'http://vocab.getty.edu/aat/300435452'

In [45]:
data['referred_to_by'][6]['classified_as'][0]['classified_as'][0]['_label']

'brief text'

In [ ]:
class Assigned(BaseModel):
    id: str
    type: str

class AssignedBy(BaseModel):
    assigned: list[Assigned] = []

class Part(BaseModel):
    assigned_by: list[AssignedBy] = []

class ProducedBy(BaseModel):
    part: list[Part] = []

class _ClassifiedAs(BaseModel):
    label: str

    @model_validator(mode="before")
    def validator(cls, data):
        return {**data, "label": data['_label']}
    

class ClassifiedAs(BaseModel):
    id: str
    classified_as: list[_ClassifiedAs] = []

class ReferredToBy(BaseModel):
    content: str
    classified_as: list[ClassifiedAs] = []
    
class Record(BaseModel):
    produced_by: ProducedBy
    referred_to_by: list[ReferredToBy] = []
    artist_id: str | None = None
    description: str | None = None

    def _get_artist_id(self):
        for part in self.produced_by.part:
            for assigned_by in part.assigned_by:
                for assigned in assigned_by.assigned:
                    if assigned.type == "Person":
                        return assigned.id

    def _get_description(self):
        for referred_to_by in self.referred_to_by:
            for classified_as in referred_to_by.classified_as:
                if classified_as.id == 'http://vocab.getty.edu/aat/300435452':
                    for _classified_as in classified_as.classified_as:
                        if _classified_as.label == "brief text":
                            self.description = referred_to_by.content
                            return referred_to_by.content

    @model_validator(mode="after")
    def validator(self):
        self.artist_id = self._get_artist_id()
        self.description = self._get_description()
        return self

record = Record.model_validate(data)
record.artist_id, record.description

NameError: name 'data' is not defined

In [12]:
artist_ids = []

with httpx.Client(follow_redirects=True) as client:
    for object_id in tqdm(df.object_id.to_list()):
        response = client.get(object_id)
        data = response.json()
        try:
            record = Record.model_validate(data)
            artist_ids.append(record.artist_id)
        except Exception as e:
            print(f"Failed to fetch data for {object_id}: {e}")

  0%|          | 0/814225 [00:00<?, ?it/s]

Failed to fetch data for https://id.rijksmuseum.nl/200100479
Failed to fetch data for https://id.rijksmuseum.nl/200100523
Failed to fetch data for https://id.rijksmuseum.nl/200100694
Failed to fetch data for https://id.rijksmuseum.nl/200100695
Failed to fetch data for https://id.rijksmuseum.nl/200100696
Failed to fetch data for https://id.rijksmuseum.nl/200100697
Failed to fetch data for https://id.rijksmuseum.nl/20010073
Failed to fetch data for https://id.rijksmuseum.nl/20010075
Failed to fetch data for https://id.rijksmuseum.nl/200100841
Failed to fetch data for https://id.rijksmuseum.nl/200100880
Failed to fetch data for https://id.rijksmuseum.nl/200100882
Failed to fetch data for https://id.rijksmuseum.nl/200100883
Failed to fetch data for https://id.rijksmuseum.nl/200100884
Failed to fetch data for https://id.rijksmuseum.nl/200100885
Failed to fetch data for https://id.rijksmuseum.nl/200100888
Failed to fetch data for https://id.rijksmuseum.nl/200100891
Failed to fetch data for h

KeyboardInterrupt: 

In [14]:
import requests
response = requests.get("https://id.rijksmuseum.nl/200100479")
response.json()

{'@context': 'https://linked.art/ns/v1/linked-art.json',
 'id': 'https://id.rijksmuseum.nl/200100479',
 'type': 'HumanMadeObject',
 'produced_by': {'type': 'Production',
  'timespan': {'type': 'TimeSpan',
   'identified_by': [{'type': 'Name',
     'content': 'ca. 1696',
     'language': [{'id': 'http://vocab.getty.edu/aat/300388256',
       'type': 'Language',
       '_label': 'Dutch'}]},
    {'type': 'Name',
     'content': 'c. 1696',
     'language': [{'id': 'http://vocab.getty.edu/aat/300388277',
       'type': 'Language',
       '_label': 'English'}]}],
   'begin_of_the_begin': '1696-01-01T00:00:00Z',
   'end_of_the_end': '1696-12-31T23:59:59Z'},
  'part': [{'type': 'Production',
    'identified_by': [{'type': 'Identifier',
      'content': '1',
      'classified_as': [{'id': 'http://vocab.getty.edu/aat/300456575',
        'type': 'Type'}]}],
    'classified_as': [{'id': 'http://vocab.getty.edu/aat/300404450',
      'type': 'Type'}]}]},
 'subject_of': [{'id': 'https://data.rijksmus

In [13]:
pd.Series(artist_ids).value_counts()

https://id.rijksmuseum.nl/21036904    26
https://id.rijksmuseum.nl/21030794    15
https://id.rijksmuseum.nl/21022595    14
https://id.rijksmuseum.nl/21018230    14
https://id.rijksmuseum.nl/21047413    11
                                      ..
https://id.rijksmuseum.nl/21014996     1
https://id.rijksmuseum.nl/21029471     1
https://id.rijksmuseum.nl/2102219      1
https://id.rijksmuseum.nl/2101272      1
https://id.rijksmuseum.nl/2103429      1
Name: count, Length: 86, dtype: int64

In [9]:
async def fetch_record(client, url):
    try:
        response = await client.get(url)
        response.raise_for_status()
        data = response.json()
        record = Record.model_validate(data)
        return record
    except httpx.HTTPError as exc:
        print(f"HTTP Error for {url}: {exc}")
        try:
            print(f"Response text: {response.text}")
        except Exception:
            pass
        return None
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return None

async def get_records(ids: list[str]):
    async with httpx.AsyncClient(follow_redirects=True) as client:
        tasks = [fetch_record(client, object_id) for object_id in ids]
        results = await tqdm_asyncio.gather(*tasks, desc="Fetching Records") # Use tqdm_asyncio.gather directly
        return results

batch_size = 10
records = []
for i in tqdm(range(100), desc="Fetching records in batches"):
    records += await get_records(df.object_id.iloc[i*batch_size:(i+1)*batch_size].to_list())

print([r.artist_id for r in records])

Fetching records in batches:   0%|          | 0/100 [00:00<?, ?it/s]

Fetching Records: 100%|██████████| 10/10 [00:05<00:00,  1.98it/s]


HTTP Error for https://id.rijksmuseum.nl/200100125: 
HTTP Error for https://id.rijksmuseum.nl/20010013: 
HTTP Error for https://id.rijksmuseum.nl/200100132: 
HTTP Error for https://id.rijksmuseum.nl/200100130: 
HTTP Error for https://id.rijksmuseum.nl/200100131: 
HTTP Error for https://id.rijksmuseum.nl/200100126: 
HTTP Error for https://id.rijksmuseum.nl/200100129: 
HTTP Error for https://id.rijksmuseum.nl/200100133: 
HTTP Error for https://id.rijksmuseum.nl/200100127: 
HTTP Error for https://id.rijksmuseum.nl/200100128: 


Fetching Records: 100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


HTTP Error for https://id.rijksmuseum.nl/200100265: 
HTTP Error for https://id.rijksmuseum.nl/200100261: 
HTTP Error for https://id.rijksmuseum.nl/200100258: 
HTTP Error for https://id.rijksmuseum.nl/200100263: 
HTTP Error for https://id.rijksmuseum.nl/200100264: 
HTTP Error for https://id.rijksmuseum.nl/200100260: 
HTTP Error for https://id.rijksmuseum.nl/200100259: 
HTTP Error for https://id.rijksmuseum.nl/20010026: 
HTTP Error for https://id.rijksmuseum.nl/200100262: 
HTTP Error for https://id.rijksmuseum.nl/200100257: 


...

Error processing https://id.rijksmuseum.nl/200100958: 1 validation error for Record
  Value error, No description found [type=value_error, input_value={'@context': 'https://lin...26050', 'type': 'Set'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


Fetching Records: 100%|██████████| 10/10 [00:05<00:00,  1.98it/s]


HTTP Error for https://id.rijksmuseum.nl/200100966: 
HTTP Error for https://id.rijksmuseum.nl/200100962: 
HTTP Error for https://id.rijksmuseum.nl/200100960: 
HTTP Error for https://id.rijksmuseum.nl/200100964: 
HTTP Error for https://id.rijksmuseum.nl/200100965: 
HTTP Error for https://id.rijksmuseum.nl/200100961: 
HTTP Error for https://id.rijksmuseum.nl/200100963: 
HTTP Error for https://id.rijksmuseum.nl/20010096: 


Fetching Records: 100%|██████████| 10/10 [00:00<00:00, 39.84it/s]


AttributeError: 'NoneType' object has no attribute 'artist_id'

In [10]:
pd.Series([r.artist_id if r is not None else None for r in records]).value_counts()

https://id.rijksmuseum.nl/21030794     13
https://id.rijksmuseum.nl/2108064       8
https://id.rijksmuseum.nl/21040861      3
https://id.rijksmuseum.nl/2103131       3
https://id.rijksmuseum.nl/21029638      2
https://id.rijksmuseum.nl/21084506      2
https://id.rijksmuseum.nl/210234        2
https://id.rijksmuseum.nl/2108277       2
https://id.rijksmuseum.nl/2105222       1
https://id.rijksmuseum.nl/21010785      1
https://id.rijksmuseum.nl/21010664      1
https://id.rijksmuseum.nl/21036016      1
https://id.rijksmuseum.nl/21019368      1
https://id.rijksmuseum.nl/21030699      1
https://id.rijksmuseum.nl/2101436       1
https://id.rijksmuseum.nl/210155764     1
https://id.rijksmuseum.nl/2106014       1
https://id.rijksmuseum.nl/2108110       1
https://id.rijksmuseum.nl/2108155       1
https://id.rijksmuseum.nl/2102402       1
https://id.rijksmuseum.nl/21012072      1
https://id.rijksmuseum.nl/2102261       1
https://id.rijksmuseum.nl/2108163       1
https://id.rijksmuseum.nl/21043307